In [1]:
# Add path candidates to sys.path to import utility / helper modules
import sys
sys.path.append('/workspace')

In [2]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [3]:
"""
Quick testing with existing data files

"""
from contextlib import redirect_stdout
from difflib import unified_diff
from io import StringIO
from pathlib import Path
from re import findall


# Run main() with data files and prints out differences between
# expected data and output
def run(main):
    # Lookup data file with directory name
    cwd = Path.cwd()
    data = cwd / f'{cwd.name}.dat'

    # If <working_directory_name>.dat is not found
    # then find any first data file in working directory
    if not data.exists():
        data = None
        for child in cwd.iterdir():
            if child.name.endswith('.dat'):
                data = child
                break

    # If there is no data file, do panic
    if data is None:
        raise FileNotFoundError('.dat file is not found')

    # Read data file and mock stdin with StringIO
    with data.open(mode='r', encoding='utf-8') as fp:
        data = fp.read()
        input = findall(r'^\[in\]([\d\w\s]*)\[out\]', data)[0]
        ifs = StringIO(input.strip())
        expected = findall(r'\[out\]([\s\S]*)$', data)[0]

    # Run program and with redirection of stdout to buffer
    with StringIO() as ofs, redirect_stdout(ofs):
        main(ifs.readline)
        result = ofs.getvalue()

    # Print differences
    sys.stdout.writelines(
        unified_diff(
            expected.lstrip().splitlines(keepends=True),
            result.lstrip().splitlines(keepends=True),
            fromfile='expected',
            tofile='result',
        )
    )

In [4]:
"""
Version 1:
- used methods: DP

"""
from functools import lru_cache


# Recurrence relation of length of :k-generation dragon curve
#
#   length(n + 1) = 2 * length(n) + 2
#   length(0) = 1
#
@lru_cache(maxsize=51)
def length(k):
    return 1 if k == 0 else (2 * length(k - 1) + 2)


# Returns :generation-th dragon curve's [:start - 1, :start - 1 + :size) slice
# It's because :start starts from 1
def solution(generation, start, size):
    start -= 1

    # Skips :skip number of symbols and returns the following symbol
    # Rules: X -> X+YF
    #        Y -> FX-Y
    #
    # G(0) = FX
    # G(1) = FX+YF
    # G(2) = FX+YF+FX-YF
    # ...
    def solve(curve, k, skip):
        # Base condition: reached target generation
        if k == 0:
            return curve[skip]

        for symbol in curve:
            if symbol in {'X', 'Y'}:
                # The length of expanded
                c = length(k)
                if skip >= c:
                    # Skip away redundant symbols
                    skip -= c
                elif symbol == 'X':
                    return solve('X+YF', k - 1, skip)
                else:  # elif symbol == 'Y':
                    return solve('FX-Y', k - 1, skip)
            elif skip > 0:
                # Constant symbols: +, -, F
                skip -= 1
            else:
                # Passed all :skip symbols
                return symbol

        return '## MUST NOT REACH ##'

    return ''.join(solve('FX', generation, pos)
                   for pos in range(start, start + size))


# Main I/O part
def main(rl):
    C = int(rl())
    for _ in range(C):
        n, p, l = map(int, rl().split())
        result = solution(n, p, l)  # nth_dragon_curve[p-1:p-1+l]
        print(result)


# Additional codes to simulate I/O
try:
    import IPython
except ImportError as _:
    # Submit env
    import sys
    main(sys.stdin.readline)
else:
    # IPython env
    run(main)

65:15: E741 ambiguous variable name 'l'


In [5]:
"""
Testing

"""
import cProfile
import traceback
import helpers as hlpr


f = hlpr.clock(hlpr.timeout(seconds=10)(solution))
test_cases = [
    ((0, 1, 2),
     'FX'),
    ((1, 1, 5),
     'FX+YF'),
    ((2, 6, 5),
     '+FX-Y'),
    ((42, 764853475, 30),
     'FX-YF-FX+YF+FX-YF-FX+YF-FX-YF-'),
    ((50, 10**9, 50),
     'YF-FX-YF-FX+YF+FX-YF-FX+YF-FX-YF-FX+YF+FX-YF+FX+YF'),
]

passed, failed = [], []
for idx, tc in enumerate(test_cases, start=1):
    case, expected = tc
    result = None
    try:
        result = f(*case)
        assert result == expected, \
            'wants {} but got {}'.format(hlpr.truncate(expected, 20), hlpr.truncate(result, 20))  # noqa: E501
    except Exception as err:
        fmt = 'Case {} FAIL: {}'
        print(fmt.format(idx, str(err)))
        traceback.print_exc()
        failed.append(idx)
    else:
        fmt = 'Case {} PASS'
        print(fmt.format(idx))
        passed.append(idx)

    print()

print()
if len(passed) == len(test_cases):
    print('All test(s) have passed')
else:
    fmt = '{} of {} test(s) failed: {}'
    print(fmt.format(len(failed), len(test_cases), ', '.join(map(str, failed))))  # noqa: E501

[0.00005790] 'solution(0, 1, 2)' → "'FX'"
Case 1 PASS

[0.00024550] 'solution(1, 1, 5)' → "'FX+YF'"
Case 2 PASS

[0.00004700] 'solution(2, 6, 5)' → "'+FX-Y'"
Case 3 PASS

[0.00095660] 'solution(42, 764853475, 30)' → "'FX-YF-FX+YF+FX-YF-FX+YF-FX-YF-'"
Case 4 PASS

[0.00236050] 'solution(50, 1000000000, 50)' → "'YF-FX-YF-FX+YF+FX-YF-FX+YF-FX-YF-FX+YF+FX-YF+FX+YF'"
Case 5 PASS


All test(s) have passed
